In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using KrylovKit
using SparseArrays
using LinearAlgebra
using Plots

using MatrixProductStateTools
using ExactDiagonalizationTools
using SymTensors
using GaussianFermions
using GutzwillerMPS

In [ ]:
include("/home/amir/work/mps/test/runtests.jl")

In [ ]:
⊗ = kron
H = xxz_hamiltonian(8)
es, vs = eigsolve(H, 1, :SR)
extended_sz(lx, l) = sparse(1.0I, 2^(l-1), 2^(l-1)) ⊗ [1.0 0; 0. -1.0] ⊗ sparse(1.0I, 2^(lx-l), 2^(lx-l))
mps = MatrixProductState(8, 2, vs[1])
szvals = [measure_1point(mps, [1.0 0.0; 0.0 -1.0], l) for l in 1:8]
@show szvals
@show norm2(mps)
data1 = measure_2point(mps, [1.0 0.0; 0.0 -1.0], [1.0 0.0; 0.0 -1.0]) 
data1[[half_measurement_index(8,1,l+1) for l=1:7]]
plot(data1[[half_measurement_index(8,1,l+1) for l=1:7]])

In [ ]:
cm = correlationmatrix(hopping_chain(8, boundary=:OBC), 4)
fgs = generate_fishmangates(cm)
mps = fishman2mps(fgs, 100)
data = measure_2point(mps, [1.0 0.0; 0.0 -1.0], [1.0 0.0; 0.0 -1.0]) 
plot(data[[half_measurement_index(8,1,l+1) for l=1:7]])

In [ ]:
mpsgutz = zipandgutzwiller!(mps, mps);

In [ ]:
@show mps_dims_are_consistent(mpsgutz)
data = measure_2point(mpsgutz, [1.0 0.0; 0.0 -1.0], [1.0 0.0; 0.0 -1.0]) 
data = real.(data ./ norm2(mpsgutz))
@show norm2(mpsgutz)
p = plot( data[[half_measurement_index(8,1,l+1) for l=1:7]] )
plot!(p,  data1[[half_measurement_index(8,1,l+1) for l=1:7]] )

In [ ]:
function zero_small!(M::AbstractArray, tol=1.e-14)
    for l in eachindex(M)
        if abs(M[l]) ≤ tol
            M[l] = 0
        end
    end
    M
end

In [ ]:
# ok perfect! Now we do this using symmetric tensors
lx = 16
H, block_states = xxz_hamiltonian(lx, mode=:U1)
es, vs = eigsolve(H, 1, :SR)
symmps = SymMatrixProductState(lx, 2, vs[1])
sym_sz = SymTensor(0, 
    (STLeg(+1, [0, 1], [1, 1]), STLeg(-1, [0, 1], [1, 1])),
    [(0, 0), (1, 1)],
    [-.5*ones(1,1), .5*ones(1,1)]
)
szvals = [measure_1point(symmps, sym_sz, l) for l in 1:8]
@show szvals
#@show norm2(mps)
measure_1point(symmps, sym_sz)
data1 = measure_2point(symmps, sym_sz, sym_sz) 
plot(data1[[half_measurement_index(lx,1,l+1) for l=1:lx-1]])

In [ ]:
lx=8
cm = correlationmatrix(hopping_chain(lx, boundary=:OBC), 4)
fgs = generate_fishmangates_sym(cm)
mps = fishman2symmps(fgs, 100)
data = measure_2point(mps, sym_sz, sym_sz) 
plot(data[[half_measurement_index(lx,1,l+1) for l=1:lx-1]])